In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import msprime
import tskit

import sys
sys.path.append("../")
from tsimpute import beagle_numba

In [2]:
N = 1_000
L = 1e6
rho = 1e-8
mu = 1e-8
ts = msprime.sim_mutations(
    msprime.sim_ancestry(
        N,
        sequence_length=L,
        discrete_genome=True,
        recombination_rate=rho,
        population_size=1e4,
    ),
    rate=mu,
)
ts

In [3]:
ne = 1e6
err_rate = 1e-4
cm = beagle_numba.convert_to_genetic_map_positions(ts.sites_position)
trans_probs = beagle_numba.get_transition_probs(cm, h=ts.num_samples, ne=ne)
mismatch_probs = beagle_numba.get_mismatch_probs(ts.num_sites, error_rate=err_rate)

In [4]:
ref_h = ts.genotype_matrix(alleles=tskit.ALLELES_ACGT)
ref_h.shape

(3238, 2000)

In [5]:
query_h = np.random.choice(a=np.arange(4, dtype=np.int32), size=ts.num_sites)

In [6]:
fwd_mat = beagle_numba.compute_forward_matrix_beaglelike(
    ref_h=ref_h,
    query_h=query_h,
    trans_probs=trans_probs,
    mismatch_probs=mismatch_probs,
    num_alleles=4,
)
fwd_mat.shape

(3238, 2000)

In [7]:
fwd_mat_refactored = beagle_numba.compute_forward_matrix(
    ref_h=ref_h,
    query_h=query_h,
    trans_probs=trans_probs,
    mismatch_probs=mismatch_probs,
    num_alleles=4,
)
fwd_mat_refactored.shape

(3238, 2000)

In [8]:
np.testing.assert_array_almost_equal(fwd_mat, fwd_mat_refactored)

In [9]:
bwd_mat = beagle_numba.compute_backward_matrix_beaglelike(
    ref_h=ref_h,
    query_h=query_h,
    trans_probs=trans_probs,
    mismatch_probs=mismatch_probs,
    num_alleles=4,
)
bwd_mat.shape

(3238, 2000)

In [10]:
bwd_mat_refactored = beagle_numba.compute_backward_matrix(
    ref_h=ref_h,
    query_h=query_h,
    trans_probs=trans_probs,
    mismatch_probs=mismatch_probs,
    num_alleles=4,
)
bwd_mat_refactored.shape

(3238, 2000)

In [11]:
state_mat = np.multiply(fwd_mat, bwd_mat)
state_mat_refactored = np.multiply(fwd_mat_refactored, bwd_mat_refactored)
np.testing.assert_array_almost_equal(state_mat, state_mat_refactored)